In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import math

GAME_DIMENSION = 10
MAX_LENGTH = 80
TRANSITION_TYPE_QUOTA = 200

print("Generating a dataset with " + MAX_LENGTH*TRANSITION_TYPE_QUOTA*16 + " samples")


def create_dataset():
    for current_length in range(MAX_LENGTH):
        current_length += 1
        # Transition quota for each snake length
        wall_up = TRANSITION_TYPE_QUOTA
        wall_right = TRANSITION_TYPE_QUOTA
        wall_down = TRANSITION_TYPE_QUOTA
        wall_left = TRANSITION_TYPE_QUOTA
        body_up = TRANSITION_TYPE_QUOTA
        body_right = TRANSITION_TYPE_QUOTA
        body_down = TRANSITION_TYPE_QUOTA
        body_left = TRANSITION_TYPE_QUOTA
        move_up = TRANSITION_TYPE_QUOTA
        move_right = TRANSITION_TYPE_QUOTA
        move_down = TRANSITION_TYPE_QUOTA
        move_left = TRANSITION_TYPE_QUOTA
        eat_up = TRANSITION_TYPE_QUOTA
        eat_right = TRANSITION_TYPE_QUOTA
        eat_down = TRANSITION_TYPE_QUOTA
        eat_left = TRANSITION_TYPE_QUOTA

        while (wall_up > 0 or wall_right > 0 or wall_down > 0 or wall_left > 0 or body_up > 0 or body_right > 0 or body_down > 0 or body_left > 0 or move_up > 0 or move_right > 0 or move_down > 0 or move_left > 0 or eat_up > 0 or eat_right > 0 or eat_down > 0 or eat_left > 0):

            snake_list = instantiate_snake(current_length)
            food_position = instantiate_food(snake_list)
            game_over = False
            
            while game_over == False:
                before_snake_list = snake_list.copy()
                before_food = food_position.copy()


def instantiate_snake(current_length):
    if current_length == 1:
        return [[random.randint(0,9), random.randint(0,9)]]
    else:
        hamiltonian_path = generate_hamiltonian_path(GAME_DIMENSION)
        start_index = random.randint(0,100 - current_length)
        end_index = start_index + current_length
        snake_list = hamiltonian_path[start_index:end_index]
        return snake_list


def snake_game():

    #Game initilisation
    snake_list = [[GAME_DIMENSION // 2, GAME_DIMENSION // 2]]
    food_position = instantiate_food(snake_list)
    game_over = False
    display_frame(snake_list,food_position)

    #Game loop
    while game_over == False:

        #Get new head position based on valid user input
        valid_input = False
        while valid_input == False:
            direction = input("Please enter a valid direction (w,a,s,d)")
            match direction:
                case 'w':
                    valid_input = True
                    new_position = [snake_list[-1][0], snake_list[-1][1] + 1]
                case 'a':
                    valid_input = True
                    new_position = [snake_list[-1][0] - 1, snake_list[-1][1]]
                case 's':
                    valid_input = True
                    new_position = [snake_list[-1][0], snake_list[-1][1] - 1]
                case 'd':
                    valid_input = True
                    new_position = [snake_list[-1][0] + 1, snake_list[-1][1]]

        #Check for collisions
        if new_position[0] < 0 or new_position[0] >= GAME_DIMENSION or new_position[1] < 0 or new_position[1] >= GAME_DIMENSION:
            game_over = True
            display_frame([],[])
            print(snake_list)
            break
        if new_position in snake_list:
            game_over = True
            display_frame([],[])
            break

        #Check for food
        snake_list.append(new_position)
        if new_position == food_position:
            food_position = instantiate_food(snake_list)
        else:
            snake_list.pop(0)
        display_frame(snake_list,food_position)



def instantiate_food(snake_list):

    #Return a valid food position
    game_grid = [[x,y] for x in range(GAME_DIMENSION) for y in range(GAME_DIMENSION)]
    valid_grid = [position for position in game_grid if position not in snake_list]
    if not valid_grid:
        return []
    else:
        return random.choice(valid_grid)

def display_frame(snake_list, food_position):

    frame = np.zeros((GAME_DIMENSION, GAME_DIMENSION, 3), dtype=np.uint8)

    # Draw the frame
    if snake_list:
        for body in snake_list:
            frame[body[1], body[0]] = [0, 255, 0]  # Green for body
        frame[snake_list[-1][1], snake_list[-1][0]] = [0, 0, 255]  # Blue for head
    if food_position:
        frame[food_position[1], food_position[0]] = [255, 0, 0]  # Red for food

    plt.imshow(frame, interpolation='nearest', origin='lower')
    plt.show()

def backbite(n, path):

    nsq = n * n
    
    # Choose one of the two endpoints at random
    itemp = random.randint(0, 1)
    
    if itemp == 0:
        # Start at beginning: path[0]
        x, y = path[0]
        xedge = (x == 0) or (x == n - 1)
        yedge = (y == 0) or (y == n - 1)
        
        if xedge and yedge:
            add_edge = random.randint(0, 2) - 2  # Corner: 1/3 acceptance
        elif xedge or yedge:
            add_edge = random.randint(0, 2) - 1  # Edge: 2/3 acceptance
        else:
            add_edge = random.randint(0, 2)      # Interior: full acceptance
        
        success = (add_edge >= 0)
        iedge = 0
        i = 3
        
        while iedge <= add_edge and i < nsq:
            dx = abs(x - path[i][0])
            dy = abs(y - path[i][1])
            
            if dx + dy == 1:  # Found an adjacent cell
                if iedge == add_edge:
                    # Reverse the walk from 0 to i-1
                    jlim = (i - 1) // 2
                    for j in range(jlim):
                        path[j], path[i - 1 - j] = path[i - 1 - j], path[j]
                iedge += 1
            
            i += max(2, dx + dy - 1)
    
    else:
        # Start at end: path[nsq-1]
        x, y = path[nsq - 1]
        xedge = (x == 0) or (x == n - 1)
        yedge = (y == 0) or (y == n - 1)
        
        if xedge and yedge:
            add_edge = random.randint(0, 2) - 2  # Corner: 1/3 acceptance
        elif xedge or yedge:
            add_edge = random.randint(0, 2) - 1  # Edge: 2/3 acceptance
        else:
            add_edge = random.randint(0, 2)      # Interior: full acceptance
        
        success = (add_edge >= 0)
        iedge = 0
        i = nsq - 4
        
        while iedge <= add_edge and i >= 0:
            dx = abs(x - path[i][0])
            dy = abs(y - path[i][1])
            
            if dx + dy == 1:  # Found an adjacent cell
                if iedge == add_edge:
                    # Reverse the walk from i+1 to nsq-1
                    jlim = (nsq - 1 - i - 1) // 2
                    for j in range(jlim):
                        path[nsq - 1 - j], path[i + 1 + j] = path[i + 1 + j], path[nsq - 1 - j]
                iedge += 1
            
            i -= max(2, dx + dy - 1)
    
    return success

def generate_hamiltonian_path(n, q=1.0):

    # Initialize path with zigzag pattern
    path = []
    for i in range(n):
        if i % 2 == 0:
            for j in range(n):
                path.append([i, j])
        else:
            for j in range(n):
                path.append([i, n - j - 1])
    
    # Calculate number of moves for equilibration
    nmoves = int(q * 10.0 * n * n * math.pow(math.log(2.0 + n), 2))
    
    # Apply backbiting moves for equilibration
    nsuccess = 0
    nattempts = 0
    
    while nsuccess < nmoves:
        success = backbite(n, path)
        if success:
            nsuccess += 1
        nattempts += 1
    
    # Apply same number of additional attempts to remove bias
    for _ in range(nattempts):
        backbite(n, path)
    
    return path